## Data Cleaning

- Data cleaning for new dataset for use in DP experiments

[Open Policing Project - Vermont](https://public.enigma.com/datasets/vermont/a34af967-056f-48e6-8838-c511d795f3ec)

Records of over 250,000 police pull overs in the state of Vermont between years 2010-2015. Note that the stop purpose information is not very granular — there are only five categories, and Open Policing has no way of identifying speeding



In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', 500)
from sklearn.linear_model import LogisticRegression
import json

In [2]:
df = pd.read_csv('../data/564afe10-629a-4317-bfa6-73f050b12f0c_Vermont.csv')
# column transforms
df['stop_date'] = pd.to_datetime(df['stop_date'])
df['hour_of_day'] = pd.to_datetime(df['stop_time']).apply(lambda x: x.hour)
df['contraband_found'] = df['contraband_found']*1

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.groupby('stop_outcome').aggregate({'stop_date':len})

,stop_date
stop_outcome,
Arrest for Violation,3255
Citation,106638
Verbal Warning,11
Warrant Arrest,76
Written Warning,170980


In [4]:
# drop NaNs
df.dropna(how = 'any', subset = ['driver_gender'], inplace = True)
df.dropna(how = 'any', subset = ['stop_outcome'], inplace = True)

In [5]:
# check that the target is not NaN
df['stop_outcome'].isna().sum()

0

In [6]:
xwalk = {}
for i in range(5):
    xwalk[i+1] = df['stop_outcome'].astype('category').cat.categories[i]
    
# save crosswalk using json
with open('../data/codes_to_cat.txt', 'w') as json_file:
    json.dump(xwalk, json_file)

In [7]:
selected_cols = [
 'hour_of_day',
#  'county_name',
 'police_department',
 'driver_gender',
 'driver_age',
 'driver_race',
 'violation',
 'search_conducted',
 'search_type',
 'contraband_found',
#  'is_arrested',
    'stop_outcome']

In [8]:
columns_to_dummy = [ 'police_department', 'driver_gender', 'driver_race',
                          'violation', 'search_type',]
new_df = pd.get_dummies(df[columns_to_dummy])

In [9]:
remaining_cols = [x for x in selected_cols if x not in columns_to_dummy]

In [10]:
# merge one hot encoded columns to numeric columns
final_df = pd.merge(df[remaining_cols], new_df, left_on = df.index, right_on = new_df.index)
# drop merge key
final_df.drop(['key_0'], axis = 1, inplace = True)
# drop NaNs
final_df.dropna(inplace = True)
#convert target col to category codes
final_df['stop_outcome'] = final_df['stop_outcome'].astype('category').cat.codes

In [11]:
final_df.head()

,hour_of_day,driver_age,search_conducted,contraband_found,stop_outcome,police_department_BRADFORD VSP,police_department_BRATTLEBORO VSP,police_department_DERBY VSP,police_department_MIDDLESEX VSP,police_department_NEW HAVEN VSP,police_department_ROCKINGHAM VSP,police_department_ROYALTON VSP,police_department_RUTLAND VSP,police_department_SHAFTSBURY VSP,police_department_ST ALBANS VSP,police_department_ST JOHNSBURY VSP,police_department_VSP HEADQUARTERS - FIELD FORCE,police_department_VSP HQ- BCI/SIU/NIU,police_department_WILLISTON VSP,driver_gender_F,driver_gender_M,driver_race_Asian,driver_race_Black,driver_race_Hispanic,driver_race_Other,driver_race_White,violation_DUI,violation_Equipment,violation_Moving violation,violation_Other,violation_Other (non-mapped),search_type_Consent Search - Probable Cause,search_type_Consent Search - Reasonable Suspicion,search_type_Warrant
0,0,22.0,False,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0
1,0,21.0,False,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0,21.0,False,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,0,18.0,False,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0
4,0,18.0,False,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0


In [12]:
final_df.to_csv('../data/alldata.csv')

---